In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import torch
import keras
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from keras.layers import TorchModuleWrapper
from keras.optimizers import Adam

In [ ]:
import json
import pandas as pd


In [ ]:
!git clone https://github.com/THU-KEG/MAVEN-ERE.git

Cloning into 'MAVEN-ERE'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 177 (delta 10), reused 4 (delta 4), pack-reused 162 (from 1)
Receiving objects: 100% (177/177), 94.03 KiB | 844.00 KiB/s, done.
Resolving deltas: 100% (94/94), done.


In [ ]:
!wget -O MAVEN_ERE.zip https://cloud.tsinghua.edu.cn/f/a7d1db6c44ea458bb6f0/?dl=1
!unzip MAVEN_ERE.zip

--2024-11-23 15:09:43--  https://cloud.tsinghua.edu.cn/f/a7d1db6c44ea458bb6f0/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 101.6.15.69, 2402:f000:1:402:101:6:15:69
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|101.6.15.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/46a7bca3-7aaa-4cc2-8031-a7773bcefe07/MAVEN_ERE.zip [following]
--2024-11-23 15:09:44--  https://cloud.tsinghua.edu.cn/seafhttp/files/46a7bca3-7aaa-4cc2-8031-a7773bcefe07/MAVEN_ERE.zip
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 200 OK
Length: 19635238 (19M) [application/zip]
Saving to: ‘MAVEN_ERE.zip’

MAVEN_ERE.zip       100%[===================>]  18.73M  7.26MB/s    in 2.6s    

2024-11-23 15:09:47 (7.26 MB/s) - ‘MAVEN_ERE.zip’ saved [19635238/19635238]

Archive:  MAVEN_ERE.zip
   creating: MAVEN_ERE/
  inflating: MAVEN_ERE/test.jsonl    
  inflating: MAVEN

In [ ]:
file_path = "/content/MAVEN_ERE/train.jsonl"
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

df = pd.DataFrame(data)



In [ ]:
data[0]['events'][0]['mention'][0]

{'id': 'd67d19cc8d3a9346e6dbea2a76a6d1d9',
 'trigger_word': 'ruled',
 'sent_id': 1,
 'offset': [38, 39]}

In [ ]:
events_data = []
for doc in data:
    doc_id = doc['id']
    for event in doc.get('events', []):
          events_data.append({
              'doc_id': doc_id,
              'event_id': event['id'],
              'event_type': event['type'],
              'event_type_id': event['type_id'],
              'mention_tw': event['mention'][0]['trigger_word'],
              'sent_id': event['mention'][0]['sent_id'],
              'offset': event['mention'][0]['offset']
            })

events_df = pd.DataFrame(events_data)

In [ ]:
#list of precondition events
events_df.set_index('event_id').loc[[item for sublist in df.set_index('id').loc['364ed14fc610df6e25a2f446e2b2d2ab']['causal_relations']['PRECONDITION'] for item in sublist]]

,doc_id,event_type,event_type_id,mention_tw,sent_id,offset
event_id,,,,,,
EVENT_040b46602889e455df7b4327137034e9,364ed14fc610df6e25a2f446e2b2d2ab,Cause_to_make_progress,137,bolster,5,"[16, 17]"
EVENT_c027e659d7fe424a0a57ecbe35b3a7f9,364ed14fc610df6e25a2f446e2b2d2ab,Conquering,21,conquer,1,"[30, 31]"
EVENT_f535eb21f8f958a04cbc69c1d1521186,364ed14fc610df6e25a2f446e2b2d2ab,Self_motion,46,Expedition,0,"[1, 2]"
EVENT_5445faf0caf78e3dcc1dc88e3554fac8,364ed14fc610df6e25a2f446e2b2d2ab,Process_end,61,end,6,"[49, 50]"
EVENT_040b46602889e455df7b4327137034e9,364ed14fc610df6e25a2f446e2b2d2ab,Cause_to_make_progress,137,bolster,5,"[16, 17]"
EVENT_0b0c5d1268cc135278489b7f0004c0c3,364ed14fc610df6e25a2f446e2b2d2ab,Achieve,127,achieve,6,"[19, 20]"
EVENT_040b46602889e455df7b4327137034e9,364ed14fc610df6e25a2f446e2b2d2ab,Cause_to_make_progress,137,bolster,5,"[16, 17]"
EVENT_931d31a625729392d27925e697762d5f,364ed14fc610df6e25a2f446e2b2d2ab,Self_motion,46,venture,4,"[2, 3]"
EVENT_f535eb21f8f958a04cbc69c1d1521186,364ed14fc610df6e25a2f446e2b2d2ab,Self_motion,46,Expedition,0,"[1, 2]"


In [ ]:
events_df.set_index('event_id').loc['EVENT_040b46602889e455df7b4327137034e9']

,EVENT_040b46602889e455df7b4327137034e9
doc_id,364ed14fc610df6e25a2f446e2b2d2ab
event_type,Cause_to_make_progress
event_type_id,137
mention_tw,bolster
sent_id,5
offset,"[16, 17]"


In [ ]:
causal_data = []
for doc in data:
    doc_id = doc['id']
    for relation_type, relations in doc.get('causal_relations', {}).items():
        for relation in relations:
            causal_data.append({
                'doc_id': doc_id,
                'relation_type': relation_type,
                'cause_event': relation[0],  # The first event ID in the relation
                'effect_event': relation[1]  # The second event ID in the relation
            })

causal_df = pd.DataFrame(causal_data)

In [ ]:
causal_events_df = events_df.set_index('event_id').loc[list(causal_df['cause_event'])]
effect_events_df = events_df.set_index('event_id').loc[list(causal_df['effect_event'])]
noncausal_events_df = events_df[~events_df['event_id'].isin(causal_events_df.index)]



In [ ]:
causal_events_df['event_type'].value_counts()


,count
event_type,
Coming_to_be,3453
Catastrophe,3250
Attack,2549
Hostile_encounter,1662
Process_start,1281
...,...
Labeling,5
Extradition,4
Emergency,4


In [ ]:
effect_events_df['event_type'].value_counts()


,count
event_type,
Killing,1522
Motion,1290
Destroying,1106
Damaging,1104
Bodily_harm,1050
...,...
Breathing,6
Ingestion,5
Renting,3


In [ ]:
noncausal_events_df['event_type'].value_counts()

,count
event_type,
Causation,2454
Process_start,2136
Motion,1477
Hostile_encounter,1421
Attack,1405
...,...
Renting,6
Risk,6
Breathing,4


In [ ]:
noncausal_causation_ids = list(noncausal_events_df[noncausal_events_df['event_type'] == 'Causation']['event_id'])

# Filter rows in causal_df
filtered_causal_df = causal_df[
    causal_df['cause_event'].isin(noncausal_causation_ids) |
    causal_df['effect_event'].isin(noncausal_causation_ids)
]

print(filtered_causal_df)

Empty DataFrame
Columns: [doc_id, relation_type, cause_event, effect_event]
Index: []


In [ ]:
filtered_df = df[
    df['id'].isin(list(noncausal_events_df[noncausal_events_df['event_type'] == 'Causation']['doc_id']))
]


In [ ]:
filtered_df['events'].iloc[0]

[{'id': 'EVENT_70824ec59e32d4f9f1edc2ba7e80ab90',
  'type': 'Motion',
  'type_id': 10,
  'mention': [{'id': 'f3a7903e62d6a0c39f6de6840f58a3da',
    'trigger_word': 'fell',
    'sent_id': 12,
    'offset': [33, 34]}]},
 {'id': 'EVENT_6f6b8673d098cd0aaa97bceeb302674c',
  'type': 'Coming_to_be',
  'type_id': 29,
  'mention': [{'id': 'ee7567c302e7ac7103cb5a262659e29c',
    'trigger_word': 'occurred',
    'sent_id': 17,
    'offset': [2, 3]}]},
 {'id': 'EVENT_86f3e66fd22e185f6c132d1c197ebd31',
  'type': 'Causation',
  'type_id': 5,
  'mention': [{'id': 'ba2e82e63765850fb1e736eead3dd8a4',
    'trigger_word': 'caused',
    'sent_id': 12,
    'offset': [15, 16]}]},
 {'id': 'EVENT_cf4b81a7ad9f39307502eb49834092f7',
  'type': 'Damaging',
  'type_id': 11,
  'mention': [{'id': 'f0f068965d53b2f3fb971965a4f303ba',
    'trigger_word': 'Damage',
    'sent_id': 16,
    'offset': [0, 1]}]},
 {'id': 'EVENT_365c60e7cad228a99ea281ea5131651c',
  'type': 'Placing',
  'type_id': 4,
  'mention': [{'id': 'bbd95

In [ ]:
filtered_df['causal_relations'].iloc[0]

{'CAUSE': [['EVENT_f444a52cf6695b576d0b7605584dfaee',
   'EVENT_cf4b81a7ad9f39307502eb49834092f7'],
  ['EVENT_f444a52cf6695b576d0b7605584dfaee',
   'EVENT_77441b0e18d134b8584dc50a20c76fd8'],
  ['EVENT_0e37d73c9901e6506ea3b779b206cdc7',
   'EVENT_cf4b81a7ad9f39307502eb49834092f7'],
  ['EVENT_f444a52cf6695b576d0b7605584dfaee',
   'EVENT_4781ef6204d0b0f193bf43751e7b7fa3'],
  ['EVENT_904c65952df2b2b276e33678dc74ba80',
   'EVENT_2ae8b11b9c00c4ae10a793356d1795c3'],
  ['EVENT_0e37d73c9901e6506ea3b779b206cdc7',
   'EVENT_77441b0e18d134b8584dc50a20c76fd8'],
  ['EVENT_70824ec59e32d4f9f1edc2ba7e80ab90',
   'EVENT_a4a2d243d5b0fc507b4582c3fcde9bc5'],
  ['EVENT_f444a52cf6695b576d0b7605584dfaee',
   'EVENT_699b3e589ac75144d73da86433379ba0']],
 'PRECONDITION': [['EVENT_0e37d73c9901e6506ea3b779b206cdc7',
   'EVENT_6f9ed94d2dc419c31a15f23a5096cce3'],
  ['EVENT_f0234afc666fc2eca2821424384998f3',
   'EVENT_a2d921171a38375bb9144633ba93aea5'],
  ['EVENT_f444a52cf6695b576d0b7605584dfaee',
   'EVENT_6f9ed94d2

In [ ]:
types=[]
for doc in data:
  for event in doc.get('events', []):
    types.append(event['type'])

In [ ]:
causal_df.index

RangeIndex(start=0, stop=36316, step=1)

In [ ]:
events_df[events_df['event_type'] == 'Causation']['doc_id']

,doc_id
54,097b86ef2a4c5d21037a4c6b47b3a165
75,097b86ef2a4c5d21037a4c6b47b3a165
76,097b86ef2a4c5d21037a4c6b47b3a165
124,ab70a3e49966caa8c35f8b27fabea3ad
181,c20868c301a1b2170bacc32d02b25a32
...,...
67839,7bb744c33ddc039deefc1ccb54928b6c
67852,42f11da5cd739248abdc64d6c66ee756
67862,42f11da5cd739248abdc64d6c66ee756
67884,8085aece72f5b46cecef0baed8327327


In [ ]:
common_doc_ids = causal_df.set_index('doc_id').index.intersection(list(events_df[events_df['event_type'] == 'Causation']['doc_id']))



causal_df.set_index('doc_id').loc[common_doc_ids]

,relation_type,cause_event,effect_event
doc_id,,,
097b86ef2a4c5d21037a4c6b47b3a165,CAUSE,EVENT_f444a52cf6695b576d0b7605584dfaee,EVENT_cf4b81a7ad9f39307502eb49834092f7
097b86ef2a4c5d21037a4c6b47b3a165,CAUSE,EVENT_f444a52cf6695b576d0b7605584dfaee,EVENT_77441b0e18d134b8584dc50a20c76fd8
097b86ef2a4c5d21037a4c6b47b3a165,CAUSE,EVENT_0e37d73c9901e6506ea3b779b206cdc7,EVENT_cf4b81a7ad9f39307502eb49834092f7
097b86ef2a4c5d21037a4c6b47b3a165,CAUSE,EVENT_f444a52cf6695b576d0b7605584dfaee,EVENT_4781ef6204d0b0f193bf43751e7b7fa3
097b86ef2a4c5d21037a4c6b47b3a165,CAUSE,EVENT_904c65952df2b2b276e33678dc74ba80,EVENT_2ae8b11b9c00c4ae10a793356d1795c3
...,...,...,...
8085aece72f5b46cecef0baed8327327,CAUSE,EVENT_541ca4e150e8238e5660ab063fc2427d,EVENT_ad3aec8ed217a99305c9fea36b7ca204
8085aece72f5b46cecef0baed8327327,PRECONDITION,EVENT_ae58aa3ff4757e1219a4d4268277b924,EVENT_6886aa85859391b3fa56070045ac1368
8085aece72f5b46cecef0baed8327327,PRECONDITION,EVENT_d7f64a2263604e051930a31e09e6d7d7,EVENT_ad3aec8ed217a99305c9fea36b7ca204


In [ ]:
len(causal_df)

36316

looks like most of the Causation event types are indeed tagged w a causal relation. Better to choose a diff indicator maybe like mention_tw.

In [ ]:
events_df[events_df['event_type'] == 'Causation']

,doc_id,event_id,event_type,event_type_id,mention_tw,sent_id,offset
54,097b86ef2a4c5d21037a4c6b47b3a165,EVENT_86f3e66fd22e185f6c132d1c197ebd31,Causation,5,caused,12,"[15, 16]"
75,097b86ef2a4c5d21037a4c6b47b3a165,EVENT_6c80346e5bf3be10bed651cce9e31ebb,Causation,5,resulted in,19,"[6, 8]"
76,097b86ef2a4c5d21037a4c6b47b3a165,EVENT_a95183aadc0236e47ab433d455527c3a,Causation,5,caused,0,"[2, 3]"
124,ab70a3e49966caa8c35f8b27fabea3ad,EVENT_f7e218ad1f6965518ea2e79b7952295f,Causation,5,cause,2,"[4, 5]"
181,c20868c301a1b2170bacc32d02b25a32,EVENT_f12af731e3f0ffbccf74b1d60af19c99,Causation,5,forced,7,"[2, 3]"
...,...,...,...,...,...,...,...
67839,7bb744c33ddc039deefc1ccb54928b6c,EVENT_a8c0fcabd65e0cc6fbb32fcd42ebcf76,Causation,5,left,6,"[27, 28]"
67852,42f11da5cd739248abdc64d6c66ee756,EVENT_b01d27f20328228fcf1b3005ba3fa803,Causation,5,due,4,"[9, 10]"
67862,42f11da5cd739248abdc64d6c66ee756,EVENT_cbe6dfbb49aa8de0aedd1e72681d450e,Causation,5,complete,5,"[10, 11]"
67884,8085aece72f5b46cecef0baed8327327,EVENT_0c15f032d82d53c418bfefcc7516fa3a,Causation,5,caused,13,"[4, 5]"


In [ ]:
set(types)

{'Achieve',
 'Action',
 'Adducing',
 'Agree_or_refuse_to_act',
 'Aiming',
 'Arranging',
 'Arrest',
 'Arriving',
 'Assistance',
 'Attack',
 'Award',
 'Bearing_arms',
 'Becoming',
 'Becoming_a_member',
 'Being_in_operation',
 'Besieging',
 'Bodily_harm',
 'Body_movement',
 'Breathing',
 'Bringing',
 'Building',
 'Carry_goods',
 'Catastrophe',
 'Causation',
 'Cause_change_of_position_on_a_scale',
 'Cause_change_of_strength',
 'Cause_to_amalgamate',
 'Cause_to_be_included',
 'Cause_to_make_progress',
 'Change',
 'Change_event_time',
 'Change_of_leadership',
 'Change_sentiment',
 'Change_tool',
 'Check',
 'Choosing',
 'Collaboration',
 'Come_together',
 'Coming_to_be',
 'Coming_to_believe',
 'Commerce_buy',
 'Commerce_pay',
 'Commerce_sell',
 'Commitment',
 'Committing_crime',
 'Communication',
 'Competition',
 'Confronting_problem',
 'Connect',
 'Conquering',
 'Containing',
 'Control',
 'Convincing',
 'Cost',
 'Create_artwork',
 'Creating',
 'Criminal_investigation',
 'Cure',
 'Damaging',


In [ ]:
df.iloc[0]['causal_relations']

{'CAUSE': [],
 'PRECONDITION': [['EVENT_c511a5c88a6b1babc55e83ec5331a6b5',
   'EVENT_f1e4e17fa7c45cac758d8fdbb44acd3c']]}

In [ ]:
df.iloc[0]['events']

[{'id': 'EVENT_1f0913dfc2f0b82fa2aea6415b6f7990',
  'type': 'Hindering',
  'type_id': 99,
  'mention': [{'id': '2d34c024d8689391090191eea4649642',
    'trigger_word': 'restrain',
    'sent_id': 8,
    'offset': [20, 21]}]},
 {'id': 'EVENT_f1e4e17fa7c45cac758d8fdbb44acd3c',
  'type': 'Cause_change_of_position_on_a_scale',
  'type_id': 37,
  'mention': [{'id': 'c1d9b45e187c19253f5ab854093d516e',
    'trigger_word': 'minimize',
    'sent_id': 10,
    'offset': [18, 19]}]},
 {'id': 'EVENT_8099531018fe4950d38842e142475f8c',
  'type': 'Statement',
  'type_id': 36,
  'mention': [{'id': 'f3e88a6b08f31fe150a7ae19fb593cb2',
    'trigger_word': 'described',
    'sent_id': 1,
    'offset': [3, 4]}]},
 {'id': 'EVENT_112472d4b74ff8c5c3c7c3979da47aa0',
  'type': 'Achieve',
  'type_id': 127,
  'mention': [{'id': 'ee7d8b2289caa5d3f842d377cac362a5',
    'trigger_word': 'achieved',
    'sent_id': 9,
    'offset': [7, 8]}]},
 {'id': 'EVENT_ea127e057c8a8a1d8b84dfa42cdf5dff',
  'type': 'Giving',
  'type_id'

In [ ]:
tws=[]
for doc in data:
  for event in doc.get('events', []):
    tws.append(event['mention'][0]['trigger_word'])


list(set(tws))